<a href="https://colab.research.google.com/github/Catatau52/Carteira_Quant/blob/main/Etapa_05_1_Calculadora_dos_fatores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mplcyberpunk

In [2]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40703 sha256=d0b7194a42a95057374df187d2e8f990c1edb71b6c0bad0efd9dafa344a991ad
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [3]:
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta  import relativedelta
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Carteira_Quant/Libs')

from resultados_factor import MakeReportResult

# Calculadora

In [7]:
import datetime
class backtest_indicators():

    def __init__(self, data_final, filtro_liquidez, lowBeta, highBeta,lowVol, highVol, indicadoresLabel, balanceamento, numero_ativos, nome_indicador = None, ordem_indicador = None,
                corretagem = 0, impacto_mercado = 0, data_inicial = None, nome_arquivo = 'backtest.pdf',
                caminho_imagens = None, caminho_dados = None, **kargs):

        self.lowBeta = lowBeta
        self.highBeta = highBeta
        self.lowVol = lowVol
        self.highVol = highVol
        self.indicadoresLabel = indicadoresLabel

        self.nome_arquivo = nome_arquivo
        self.caminho_imagens = caminho_imagens
        self.caminho_dados = caminho_dados

        if nome_indicador == None:

            self.indicadores = kargs

        else:

            self.indicadores = {'carteira1': {
                                            'indicadores': {
                                                nome_indicador: {'caracteristica': ordem_indicador}},
                                            'peso': 1
                                        }}


        self.balanceamento = balanceamento
        self.liquidez = filtro_liquidez

        try:

            data_inicial = datetime.datetime.strptime(data_inicial, "%Y-%m-%d").date()
            data_final = datetime.datetime.strptime(data_final, "%Y-%m-%d").date()

        except:

            data_final = datetime.datetime.strptime(data_final, "%Y-%m-%d").date()

        self.data_inicial = data_inicial
        self.data_final = data_final
        self.numero_ativos = numero_ativos
        self.corretagem = corretagem
        self.impacto_mercado = impacto_mercado
        self.dinheiro_inicial = 10000

        os.chdir(caminho_dados)


    def pegando_dados(self):

        cotacoes = pd.read_parquet('cotacoes.parquet')
        cotacoes['data'] = pd.to_datetime(cotacoes['data']).dt.date
        cotacoes['ticker'] = cotacoes['ticker'].astype(str)
        self.cotacoes = cotacoes.sort_values('data', ascending=True)

        volume_mediano = pd.read_parquet('indicador_volume_mediano.parquet')
        volume_mediano['data'] = pd.to_datetime(volume_mediano['data']).dt.date
        volume_mediano['ticker'] = volume_mediano['ticker'].astype(str)
        volume_mediano = volume_mediano[['data', 'ticker', 'valor']]
        volume_mediano.columns = ['data', 'ticker', 'volume']

        betas = pd.read_parquet('indicador_beta_756.parquet') #indicador_beta_756
        betas['data'] = pd.to_datetime(betas['data']).dt.date
        betas['ticker'] = betas['ticker'].astype(str)
        betas = betas[['data', 'ticker', 'valor']]
        betas.columns = ['data', 'ticker', 'beta']

        vol = pd.read_parquet('vol_mensal.parquet')
        vol['data'] = pd.to_datetime(vol['data']).dt.date
        vol['ticker'] = vol['ticker'].astype(str)
        vol = vol[['data', 'ticker', 'valor']]
        vol.columns = ['data', 'ticker', 'vol_mensal']

        lista_dfs = []

        lista_dfs.append(self.cotacoes)
        lista_dfs.append(volume_mediano)
        lista_dfs.append(betas)
        lista_dfs.append(vol)
        lista_indicadores_sem_rep = []

        for carteira in self.indicadores.values():
            indicadores = carteira['indicadores']

            for indicador in indicadores.keys():

                if indicador in lista_indicadores_sem_rep:

                    pass

                else:

                    lista_indicadores_sem_rep.append(indicador)

                    lendo_indicador = pd.read_parquet(f'{indicador}.parquet')
                    lendo_indicador['data'] = pd.to_datetime(lendo_indicador['data']).dt.date
                    lendo_indicador['ticker'] = lendo_indicador['ticker'].astype(str)
                    lendo_indicador['valor'] = lendo_indicador['valor'].astype(float)
                    lendo_indicador = lendo_indicador[['data', 'ticker', 'valor']]
                    lendo_indicador.columns = ['data', 'ticker', indicador]
                    lista_dfs.append(lendo_indicador)

        df_dados = lista_dfs[0]
        #print(lista_dfs)
        for df in lista_dfs[1:]:
            df_dados = pd.merge(df_dados, df,  how='inner', left_on=['data', 'ticker'], right_on=['data', 'ticker'])

        self.df_dados = df_dados.dropna()
        #df_dados.to_csv('/content/df_dados.csv', index=True, header = True)

    def filtrando_datas(self):

        df_dados = self.df_dados

        if self.data_inicial != None:
            df_dados = df_dados[df_dados['data'] >= self.data_inicial]

        else:
            df_dados = df_dados[df_dados['data'] >= (min(df_dados['data']) + relativedelta(months=+2))]

        df_dados = df_dados[df_dados['data'] < self.data_final]

        self.pegando_dias_das_carteiras(df = df_dados)
        df_dados = self.df_dados[self.df_dados['data'].isin(self.periodos_de_dias)]

        self.df_dados = df_dados
        #df_dados.to_csv('/content/df_dados.csv', index=True, header = True)
        #print(df_dados)

    def pegando_dias_das_carteiras(self, df):

        datas_disponiveis = np.sort(df['data'].unique())
        self.periodos_de_dias = [datas_disponiveis[i] for i in range(0, len(datas_disponiveis), self.balanceamento)]

    def criando_carteiras(self):

        df_dados = self.df_dados
        df_dados = df_dados[df_dados['volume'] > self.liquidez]
        df_dados = df_dados[(df_dados['beta'] >= self.lowBeta) & (df_dados['beta'] <= self.highBeta)]
        df_dados = df_dados[(df_dados['vol_mensal'] >= self.lowVol) & (df_dados['vol_mensal'] <= self.highVol)]
        # Mantém o maior volume se as 4 primeiras letras do ticker forem iguais
        df_dados = df_dados.assign(TICKER_PREFIX = df_dados['ticker'].str[:4])
        df_dados = df_dados.loc[df_dados.groupby(['data', 'TICKER_PREFIX'])['volume'].idxmax()]
        df_dados = df_dados.drop('TICKER_PREFIX', axis = 1)


        lista_df_carteiras = []

        for nome_carteira, carteira in self.indicadores.items():
            df_carteiras = df_dados.copy()
            df_carteiras[f'RANK_FINAL_{nome_carteira}'] = 0
            indicadores = carteira['indicadores']

            for indicador, ordem in indicadores.items():

                crescente_condicao = (ordem['caracteristica'] == 'crescente')
                # Crie os rankings para os indicadores
                df_carteiras[f'{indicador}_RANK_{nome_carteira}'] = df_carteiras.groupby('data')[indicador].rank(ascending = crescente_condicao)
                df_carteiras[f'RANK_FINAL_{nome_carteira}'] = df_carteiras[f'RANK_FINAL_{nome_carteira}'] + df_carteiras[f'{indicador}_RANK_{nome_carteira}']

            df_carteiras[f'posicao_{nome_carteira}'] = df_carteiras.groupby('data')[f'RANK_FINAL_{nome_carteira}'].rank()
            portfolio = df_carteiras[df_carteiras[f'posicao_{nome_carteira}'] <= self.numero_ativos]
            portfolio = portfolio.assign(peso = carteira['peso']/(portfolio.groupby('data').transform('size')))
            lista_df_carteiras.append(portfolio)

        carteira_por_periodo = pd.concat(lista_df_carteiras, ignore_index=True)

        ######################### PORTFOLIO HOLDER #####################################
        carteira_por_periodo = carteira_por_periodo.sort_values('data', ascending=True)
        carteira_por_periodo = carteira_por_periodo.drop(['preco_abertura', 'preco_fechamento', 'preco_maximo','preco_medio', 'preco_minimo', 'quantidade_negociada',
                                                          'quantidade_negocios','volume_negociado', 'fator_ajuste','preco_fechamento_ajustado', 'preco_abertura_ajustado',
                                                          'preco_maximo_ajustado', 'preco_medio_ajustado','preco_minimo_ajustado', 'volume']#, 'beta', 'vol_mensal',]
                                                          , axis = 1)
        #carteira_por_periodo.to_csv('/content/drive/MyDrive/Colab Notebooks/Carteira_Quant/backtest/carteira_fundamentos.csv', index=False, header = True)
        #carteira_por_periodo.to_csv('/content/drive/MyDrive/Colab Notebooks/Carteira_Quant/backtest/carteira_indicadores.csv', index=False, header = True)
        ######################### PORTFOLIO HOLDER #####################################

        carteira_por_periodo = carteira_por_periodo.sort_values(['data','posicao_carteira1'], ascending=True)[['data', 'ticker', 'peso','posicao_carteira1']]
        carteira_por_periodo = carteira_por_periodo.groupby(['data', 'ticker'])['peso','posicao_carteira1'].sum()
        self.carteira_por_periodo = carteira_por_periodo.reset_index()

        self.carteira_por_periodo.to_csv('/content/carteira.csv', index=True, header = True)
        print(self.carteira_por_periodo.tail(20))

    def calculando_retorno_diario(self):

        cotacoes = self.cotacoes[(self.cotacoes['data'] >= self.carteira_por_periodo.iloc[0, 0]) &
                                       (self.cotacoes['data'] <= self.data_final)]

        datas_carteira = cotacoes['data'].unique()

        df_retornos = pd.DataFrame(columns=['data', 'dinheiro', 'numero_trade'], index = list(range(0, len(datas_carteira))))

        carteira = 0

        df_retornos.iloc[1, 0] = self.carteira_por_periodo.iloc[1, 0] #segunda data (a primeira foi pra gerar o sinal)
        df_retornos.iloc[1, 1] = self.dinheiro_inicial
        df_retornos.iloc[1, 2] = carteira

        cotacoes = cotacoes.assign(var_fin = cotacoes.groupby('ticker')['preco_fechamento_ajustado'].diff())

        retorno_fin = cotacoes[['data', 'ticker', 'var_fin']]

        carteiras = self.carteira_por_periodo.copy()
        datas_rebalanceamento = carteiras['data'].unique()

        cotacoes_rebalanceamento = cotacoes[['ticker', 'data', 'preco_fechamento_ajustado']]

        retorno_fin.set_index(["data", "ticker"], inplace=True)
        carteiras.set_index(["data", "ticker"], inplace=True)
        cotacoes_rebalanceamento.set_index(["data", "ticker"], inplace=True)

        for i, data in enumerate(datas_carteira):

            if i not in [0, 1]:

                retorno_fin_dia = retorno_fin.loc[data]

                var_patrimonio_no_dia = (carteira_vigente["quantidade_acoes"] * retorno_fin_dia["var_fin"]).sum()
                df_retornos.iloc[i, 0] = data
                df_retornos.iloc[i, 1] = df_retornos.iloc[i - 1, 1] # Inicializando com o valor do dia anterior
                df_retornos.iloc[i, 1] += var_patrimonio_no_dia  # Agora a operação de adição deve funcionar corretamente
                df_retornos.iloc[i, 2] = carteira

            if data in datas_rebalanceamento:
                carteira_na_data = carteiras.loc[data].copy()
                trocar_carteira = True
                delay = 0

            if trocar_carteira:

                if delay == 0:

                    delay = delay + 1 #eu vou simular que eu só compro as açoes no preço de fechamento do dia seguinte.

                else:

                    carteira_na_data["dinheiro_por_acao"] = (carteira_na_data["peso"] * df_retornos.iloc[i, 1]) * (1 - self.corretagem) * (1 - self.impacto_mercado)
                    cotacoes_na_data = cotacoes_rebalanceamento.loc[data]
                    carteira_vigente = pd.merge(carteira_na_data, cotacoes_na_data, left_index=True, right_index=True)
                    carteira_vigente["quantidade_acoes"] = carteira_vigente["dinheiro_por_acao"] / carteira_vigente["preco_fechamento_ajustado"]
                    carteira += 1
                    trocar_carteira = False

        df_retornos = df_retornos.assign(retorno = df_retornos['dinheiro'].pct_change())
        df_retornos = df_retornos.drop(0, axis = 0)

        self.df_retornos = df_retornos

        #self.df_retornos.to_csv('/content/df_retornos.csv', index=True, header = True)
        #print (df_retornos)


    def make_report(self):

        self.carteira_por_periodo = self.carteira_por_periodo.set_index('data')

        MakeReportResult(df_trades=self.df_retornos, df_carteiras=self.carteira_por_periodo,
                         caminho_imagens=self.caminho_imagens, caminho_benchmarks= self.caminho_dados, nome_arquivo=self.nome_arquivo,
                         filtro_liquidez=self.liquidez, lowBeta=self.lowBeta, highBeta=self.highBeta, indicadoresLabel = self.indicadoresLabel,
                         lowVol = self.lowVol, highVol = self.highVol)



if __name__ == "__main__":



    dicionario_carteira = {
        'carteira1': {
                'indicadores': {
                    ############### INDICADORES TÉCNICOS ###############

                    # 'mm_7_40': {'caracteristica': 'decrescente'}, #
                    # 'mm_9_21': {'caracteristica': 'decrescente'},
                    # 'mm_13_26': {'caracteristica': 'decrescente'},
                    # 'mm_13_48': {'caracteristica': 'decrescente'},
                    # 'mm_14_40': {'caracteristica': 'decrescente'},
                    # 'mm_20_100': {'caracteristica': 'decrescente'},
                    # 'mm_20_50': {'caracteristica': 'decrescente'},
                    # 'mm_20_80': {'caracteristica': 'decrescente'}, #
                    # 'mm_50_200': {'caracteristica': 'decrescente'},

                    'momento_1_meses': {'caracteristica': 'decrescente'},##
                    # #'momento_2_meses': {'caracteristica': 'decrescente'},#
                    # #'momento_3_meses': {'caracteristica': 'decrescente'},
                    # #'momento_6_meses': {'caracteristica': 'decrescente'},
                    # #'momento_12_meses': {'caracteristica': 'decrescente'},

                    # #'rsi_14': {'caracteristica': 'crescente'},

                    # # ############### INDICADORES FUNDAMENTALISTA ###############

                    'ebit_divliq': {'caracteristica': 'decrescente'}, ##
                    'EV_EBIT': {'caracteristica': 'decrescente'}, ##
                    'My_p_vp': {'caracteristica': 'crescente'}, ##
                    'p_sr': {'caracteristica': 'crescente'}, ##
                    #'indicador_EV_Div': {'caracteristica': 'decrescente'},
                    #'indicador_p_ebit': {'caracteristica': 'decrescente'},
                    #'P_L': {'caracteristica': 'crescente'},
                    #'L_P': {'caracteristica': 'decrescente'},
                    #'ValorDeMercado': {'caracteristica': 'crescente'}, # Aumenta expectativa de retorno, porém, aumenta a volatilidade
                    #'dividend_yield': {'caracteristica': 'crescente'},

                    # ############### CRITÉRIOS PARA CARTEIRA HOLDER ###############
                    # 'ROE': {'caracteristica': 'decrescente'}, # retorno sobre o patrimonio
                    # #'ROIC': {'caracteristica': 'decrescente'},
                    # 'lucro_Receita': {'caracteristica': 'decrescente'}, #
                    # 'LPA': {'caracteristica': 'decrescente'}, #

                    # 'ebit_divliq_2': {'caracteristica': 'decrescente'}, #INSERIR BANCOS NO DF
                    # # #'pl_db': {'caracteristica': 'decrescente'},

                    # #'EV_EBIT': {'caracteristica': 'decrescente'},

                    # 'My_p_vp': {'caracteristica': 'crescente'}, #
                    # 'p_sr': {'caracteristica': 'crescente'}, #
                    # #'momento_1_meses': {'caracteristica': 'decrescente'},#
                    # 'L_P': {'caracteristica': 'decrescente'},




                },
                'peso': 1

        },
        }

    nome_pdf = ''

    for nome_carteira, carteira in dicionario_carteira.items():

            nome_pdf = nome_pdf + nome_carteira + "_peso" + str(carteira['peso']).replace(".", "") + "_"

            indicadores = carteira['indicadores']

            for indicador, ordem in indicadores.items():

                nome_pdf = nome_pdf + indicador + "_"

    balanceamento = 21 #21 (10Ativos), #20 (8Ativos)
    filtro_liquidez = 1 # 1
    numero_ativos = 10 # 10
    ### NEW ###
    lowBeta = 0  # 0
    highBeta = 1.6 # 1.6
    lowVol = 0   # 0
    highVol = 0.23 # 0.23
    indicadoresLabel = "MMT_1M, Ebit/DivLiq, EV/Ebit, P/VPA, P/SR" #MM_7x40,MktCap

    '''

    Quanto dinheiro o meu modelo aguenta?

    Qual é o meu filtro de liquidez? 1M
    Qual é o número de ativos da carteira? 10
    Qual é a % de cada ativo? 10%
    Quantos dias eu quero demorar pra comprar o modelo? 5
    Quanto eu aceito negociar do volume diário de uma ação por dia? 20%

    1M * 0.2% = 200k/0.1 = 2M * 5 = 10M

    (Filtro liquidez * % do volume negociado * dias que demora pra comprar o modelo)/% em cada ativo = Capacity

    Capacity = 10M

    '''

    nome_pdf = nome_pdf + str(balanceamento) + '_' + str(filtro_liquidez) + "M_" + str(numero_ativos) + \
    "A"+ str(lowBeta) + "_B_" + str(highBeta) + ".pdf"
    print(nome_pdf)
    nome_pdf = 'teste.pdf' #2013-01-25 #2018-01-01 #'2017-12-28'
    backtest = backtest_indicators(data_inicial= '2023-01-01', data_final="2023-12-31",filtro_liquidez=(filtro_liquidez * 1000000),
                                   lowBeta = lowBeta, highBeta = highBeta, indicadoresLabel = indicadoresLabel, lowVol = lowVol, highVol = highVol,
                                   balanceamento=balanceamento, numero_ativos=numero_ativos, corretagem=0.01,
                                   caminho_dados=r'/content/drive/MyDrive/Colab Notebooks/Carteira_Quant/base_dados_br',
                                   caminho_imagens=r'/content/drive/MyDrive/Colab Notebooks/Carteira_Quant/imagens',
                                   nome_arquivo=fr'/content/drive/MyDrive/Colab Notebooks/Carteira_Quant/backtest/'+nome_pdf, **dicionario_carteira)


    backtest.pegando_dados()
    backtest.filtrando_datas()
    backtest.criando_carteiras()
    backtest.calculando_retorno_diario()
    backtest.make_report()


carteira1_peso1_momento_1_meses_ebit_divliq_EV_EBIT_My_p_vp_p_sr_21_1M_10A0_B_1.6.pdf


<ipython-input-7-a2cea0d9b973>:183: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  carteira_por_periodo = carteira_por_periodo.groupby(['data', 'ticker'])['peso','posicao_carteira1'].sum()


           data ticker  peso  posicao_carteira1
100  2023-11-03  BRAP4   0.1                8.0
101  2023-11-03  CEAB3   0.1                4.0
102  2023-11-03  CRFB3   0.1                5.0
103  2023-11-03  ETER3   0.1               10.0
104  2023-11-03  EVEN3   0.1                1.0
105  2023-11-03  GUAR3   0.1                3.0
106  2023-11-03  JBSS3   0.1                6.5
107  2023-11-03  LJQQ3   0.1                2.0
108  2023-11-03  LWSA3   0.1                6.5
109  2023-11-03  PGMN3   0.1                9.0
110  2023-12-05  ANIM3   0.1                7.0
111  2023-12-05  CEAB3   0.1                5.0
112  2023-12-05  COGN3   0.1               10.0
113  2023-12-05  EZTC3   0.1                8.0
114  2023-12-05  GUAR3   0.1                2.5
115  2023-12-05  LJQQ3   0.1                4.0
116  2023-12-05  LREN3   0.1                6.0
117  2023-12-05  MRFG3   0.1                1.0
118  2023-12-05  PETZ3   0.1                9.0
119  2023-12-05  PFRM3   0.1            

In [ ]:
# 1245  2023-09-25  BRAP4  0.100000                4.0
# 1246  2023-09-25  CAML3  0.100000                8.0
# 1247  2023-09-25  CSNA3  0.100000                9.0
# 1248  2023-09-25  EVEN3  0.100000                3.0
# 1249  2023-09-25  GOAU4  0.100000                5.0
# 1250  2023-09-25  MRFG3  0.100000                1.0
# 1251  2023-09-25  MYPK3  0.100000               10.0
# 1252  2023-09-25  PETZ3  0.100000                6.0
# 1253  2023-09-25  USIM5  0.100000                2.0
# 1254  2023-09-25  VIVT3  0.100000                7.0
# 1254  2023-11-03  BRAP4   0.1                6.0
# 1255  2023-11-03  CEAB3   0.1                5.0
# 1256  2023-11-03  CRFB3   0.1                4.0
# 1257  2023-11-03  EMBR3   0.1                8.5
# 1258  2023-11-03  EVEN3   0.1                1.0
# 1259  2023-11-03  JBSS3   0.1                3.0
# 1260  2023-11-03  LJQQ3   0.1                2.0
# 1261  2023-11-03  LWSA3   0.1                8.5
# 1262  2023-11-03  MRFG3   0.1               10.0
# 1263  2023-11-03  PGMN3   0.1                7.0
# 691  2023-12-04  ANIM3   0.1                7.0
# 692  2023-12-04  CEAB3   0.1                4.0
# 693  2023-12-04  COGN3   0.1                9.5
# 694  2023-12-04  EZTC3   0.1                8.0
# 695  2023-12-04  LJQQ3   0.1                3.0
# 696  2023-12-04  LREN3   0.1                5.0
# 697  2023-12-04  MEAL3   0.1                9.5
# 698  2023-12-04  MRFG3   0.1                1.0
# 699  2023-12-04  PETZ3   0.1                2.0
# 700  2023-12-04  PFRM3   0.1                6.0



In [ ]:
'ROE': {'caracteristica': 'decrescente'}, # retorno sobre o patrimonio
                    #'ROIC': {'caracteristica': 'decrescente'},
                    'lucro_Receita': {'caracteristica': 'decrescente'}, #
                    'LPA': {'caracteristica': 'decrescente'}, #

                    'ebit_divliq_2': {'caracteristica': 'decrescente'}, #INSERIR BANCOS NO DF
                    # #'pl_db': {'caracteristica': 'decrescente'},

                    #'EV_EBIT': {'caracteristica': 'decrescente'},

                    'My_p_vp': {'caracteristica': 'crescente'}, #
                    'p_sr': {'caracteristica': 'crescente'}, #
                    #'momento_1_meses': {'caracteristica': 'decrescente'},#
                    'L_P': {'caracteristica': 'decrescente'},

'momento_1_meses': {'caracteristica': 'decrescente'},##
                    # # ############### INDICADORES FUNDAMENTALISTA ###############
                    'ebit_divliq': {'caracteristica': 'decrescente'}, ##
                    'EV_EBIT': {'caracteristica': 'decrescente'}, ##
                    'My_p_vp': {'caracteristica': 'crescente'}, ##
                    'p_sr': {'caracteristica': 'crescente'}, ##

In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/Carteira_Quant/base_dados_br/ebit_divliq.parquet')
df
df.loc[df['ticker'] == 'ITUB4']
#df.loc[df['valor'] > 9000]

,data,ticker,valor
